In [77]:
import pennylane as qml
import sympy as sp
import numpy as np

from sympy import symbols
from sympy.matrices import Matrix

In [46]:
teta0, teta1, teta2 = symbols('teta0 teta1 teta2')

Rz_teta0 = Matrix([
    [sp.exp(-1j*teta0/2), 0],
    [0, sp.exp(1j*teta0/2)],
])

Ry_teta1 = Matrix([
    [sp.cos(teta1/2), -sp.sin(teta1/2)],
    [sp.sin(teta1/2), sp.cos(teta1/2)],
])

Rz_teta2 = Matrix([
    [sp.exp(-1j*teta2/2), 0],
    [0, sp.exp(1j*teta2/2)],
])

U = Rz_teta0 * Ry_teta1 * Rz_teta2
U

Matrix([
[exp(-0.5*I*teta0)*exp(-0.5*I*teta2)*cos(teta1/2), -exp(-0.5*I*teta0)*exp(0.5*I*teta2)*sin(teta1/2)],
[ exp(0.5*I*teta0)*exp(-0.5*I*teta2)*sin(teta1/2),   exp(0.5*I*teta0)*exp(0.5*I*teta2)*cos(teta1/2)]])

Let's generate arbitrary unitary matrix

In [59]:
from qiskit.quantum_info import random_unitary, Operator

num_qubits = 1

V_op = random_unitary(2 ** num_qubits)
print("Is the matrix unitary?", V_op.is_unitary())
V = V_op.to_matrix()
V

Is the matrix unitary? True


array([[-0.76792859-0.60901592j,  0.15547468-0.12334062j],
       [-0.12969632+0.15021368j,  0.57657071+0.79257866j]])

In [28]:
V00 = V[0][0]
V01 = V[0][1]
V10 = V[1][0]
V11 = V[1][1]

In [65]:
np.abs(V00), np.abs(V01), 2 * np.arcsin(np.abs(V01))

(0.6595491219794227, 0.7516614634901624, 1.701155131552466)

In [63]:
beta = 2 * np.arccos(np.abs(V00)) if np.abs(V00) >= np.abs(V01) else 2 * np.arcsin(np.abs(V01))
beta

1.701155131552466

In [66]:
alpha = np.arctan2(np.imag(V11 / np.cos(beta)), np.real(V11 / np.cos(beta))) + np.arctan2(np.imag(V10 / np.sin(beta)), np.real(V10 / np.sin(beta)))
alpha

5.128402082749012

In [67]:
gamma = np.arctan2(np.imag(V11 / np.cos(beta)), np.real(V11 / np.cos(beta))) - np.arctan2(np.imag(V10 / np.sin(beta)), np.real(V10 / np.sin(beta)))
gamma

0.8730329206403469

In [68]:
U_decomposed = np.array(U.subs({teta0: alpha, teta1: beta, teta2: gamma })).astype(np.complex128)
U_decomposed, V

(array([[-0.6530153 -0.09260706j,  0.39728855+0.63808836j],
        [-0.39728855+0.63808836j, -0.6530153 +0.09260706j]]),
 array([[-0.76792859-0.60901592j,  0.15547468-0.12334062j],
        [-0.12969632+0.15021368j,  0.57657071+0.79257866j]]))

In [69]:
print("Is the matrix unitary?", Operator(U_decomposed).is_unitary())

Is the matrix unitary? True


As you can see, we have a mismatch between all elements of a decomposed matrix and original matrix V, by somehow redistributing complex phases in a different way compared to the original matrix, leading to differences in the real and imaginary components of the amplitudes. Also probably, the decomposition might not fully capture the rotational invariance of the original matrix, leading to discrepancies in the distribution of amplitudes.

In [74]:
print(np.abs(U_decomposed[0][0]), np.abs(V00))
print(np.abs(U_decomposed[0][1]), np.abs(V01))
print(np.abs(U_decomposed[1][0]), np.abs(V10))
print(np.abs(U_decomposed[1][1]), np.abs(V11))

0.6595491219794226 0.6595491219794227
0.7516614634901624 0.7516614634901624
0.7516614634901624 0.7516614634901625
0.6595491219794226 0.6595491219794226


In [82]:
zero_state = np.array([1, 0])
state0, state1 = np.dot(V, zero_state), np.dot(U_decomposed, zero_state)
print(state0, state1)

[-0.76792859-0.60901592j -0.12969632+0.15021368j] [-0.6530153 -0.09260706j -0.39728855+0.63808836j]


In [83]:
qml.math.fidelity(state0=np.outer(state0, np.conj(state0)), state1=np.outer(state1, np.conj(state1)))

0.6196327316414841